## 멜론 가사 수집 (장르별 - 인디음악) 정적 스크래핑 

In [ ]:
# url 
# https://www.melon.com/genre/song_list.htm?gnrCode=GN0500
# https://www.melon.com/song/detail.htm?songId=38427225

### 1. 환경 설정 

In [5]:
# 1. 필요 라이브러리 추가 
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os
from tqdm import tqdm
import time
import random

### 2. 데이터 불러오기 

In [4]:
# request 를 사용하여 데이터 수집할 화면 가져오기 
headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36')
}

gnr_url = "https://www.melon.com/genre/song_list.htm"

params = {
    'gnrCode': 'GN0500',
}

response = requests.get(gnr_url, params=params, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
song_list = soup.select('.wrap_song_info')

In [37]:
song_titles = []

# 모든 곡 정보를 포함하는 요소 선택
# songs = meta.select('.wrap_song_info')

# 각 곡 정보에서 곡 제목 추출
for song in song_list:
    title_element = song.select_one('.ellipsis.rank01 a')  # 곡 제목 선택
    if title_element:  # 요소가 존재할 경우
        song_titles.append(title_element.text.strip())  # 제목을 리스트에 추가

# 결과 출력
for title in song_titles:
    print(title)


어떻게든 이별
us
보통의 이야기
엔딩샷 (Ending Shot)
If
잘자요
유영
Our Time
첫눈에 반해버린 찐따
널 보고 싶지 않은 밤
우리의 사랑
당신은 날 가르치지 않았죠
Delete
Fade Away
Mom said kimchi stew is delicious.
Runnin' down the street
동백
너에게
몰라
듣는 건 너의 책임
우리의 겨울이었던
Call you my name (Feat Jae-ho)
100%
Wrong Person, Right Timing
I've chosen you
잠깐 멈춰 바라볼 때
Caution Tape (Feat. Jinee) (prod. by 1024)
새벽
나방은 빛으로 뛰어든다
Moment
KeshiA (feat. JUNGL$)
아버지가 되었어요
제자리
Never Give Up
Airport
기억할게요
Meaningless
숨 (Remastered)
그림자 (feat. 고무성)
쉬운 이별
EASYYYY
졸업하는 너희에게
나를 찾아서
DUPLICITY
이제는 애기할게
Love for
엄마
바보같지
썩은그물
blue [remastered]


In [45]:
song_titles = []

# 모든 곡 정보를 포함하는 요소 선택
songs = meta.select('.wrap_song_info')

# 각 곡 정보에서 곡 제목 추출
for song in songs:
    title_element = song.select_one('.ellipsis.rank01 a')  # 곡 제목 선택
    if title_element:  # 요소가 존재할 경우
        title = title_element.text.strip()  # 제목 가져오기
        href = title_element['href']  # href 속성 가져오기
        song_titles.append((title, href))  # 제목과 href를 튜플로 리스트에 추가

# 결과 출력
for title, href in song_titles:
    print(f"곡 제목: {title}, 링크: {href}")

print(song)

<div class="wrap_song_info">
<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('25260101',38427225);" title="어떻게든 이별 재생">어떻게든 이별</a>
</span></div><br/>
<div class="ellipsis rank02">
<a href="javascript:melon.link.goArtistDetail('105031');" title="참깨와 솜사탕 - 페이지 이동">참깨와 솜사탕</a><span class="checkEllipsis" style="display:none"><a href="javascript:melon.link.goArtistDetail('105031');" title="참깨와 솜사탕 - 페이지 이동">참깨와 솜사탕</a></span>
</div>
</div>


In [54]:
# 데이터프레임 초기화
# columns = ['chartDate', 'rank', 'title', 'singer', 'album_name', 'release_date', 'genre', 'lyric', 'composer', 'lyricist', 'arranger']
# 곡 제목 title
# 가사 lylics
# 아티스트 artist
# 장르 ganre
# 발매일 date
# 좋아요 like
columns = ['title', 'artist', 'ganre', 'release_date', 'like_cnt', 'lylics']
song_data = pd.DataFrame(columns=columns)

# tqdm 라이브러리로 진행 상황 바 표시
for i, meta in tqdm(enumerate(song_list, 1), total=len(song_list), desc="Processing songs"):
    rank = i

    # 모든 곡 정보를 포함하는 요소 선택
    songs = meta.select('.wrap_song_info')

    # 각 곡 정보에서 곡 제목 추출
    for song in song_list:
        title_element = song.select_one('.ellipsis.rank01 a')  # 곡 제목 선택
        if title_element:  # 요소가 존재할 경우
            # song_titles.append(title_element.text.strip())  # 제목을 리스트에 추가
            title = title_element.text.strip()
            href = title_element['href']  # href 속성 가져오기
            # 정규 표현식을 사용하여 곡 ID 추출
            match = re.search(r"playSong\('(\d+)',(\d+)\)", href)
            # if match:
            song_id = match.group(2)  # 두 번째 그룹이 곡 ID
            song_url = 'https://www.melon.com/song/detail.htm?songId=' + song_id

            response = requests.get(song_url, params=params, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 가수
            singer_html = soup.select('.wrap_info .artist a')
            singer_s = ', '.join([html['title'] for html in singer_html if html['title']]) if singer_html else 'Various Artists'

            # 앨범명
            # album_name = soup.select('.list dd')[0].get_text(strip=True)

            # 발매날짜
            release_date = soup.select('.list dd')[1].get_text(strip=True)

            # 장르
            genre = soup.select('.list dd')[2].get_text(strip=True)

            # 좋아요 
            # <span id="d_like_count" class="cnt">44</span>
            # like_cnt = soup.select('.cnt').get_text(strip=True)

            # 예시 코드
            like_count_element = meta.select_one('#d_like_count')  # ID로 요소 선택
            if like_count_element:  # 요소가 존재할 경우
                like_cnt = like_count_element.text.strip()  # 텍스트 가져오기 및 공백 제거
            else:
                like_cnt = 0


            # 가사
            lyric = '없음'
            lyric_html = soup.select_one('.section_lyric .wrap_lyric .lyric')
            if lyric_html:
                lyric = lyric_html.get_text(strip=True, separator='\n')

            row = pd.Series([title, singer_s, genre, release_date, like_cnt,  lyric], index=song_data.columns)
            song_data = pd.concat([song_data, pd.DataFrame([row])], ignore_index=True)

            # 1초에서 5초 사이의 랜덤한 시간 선택
            random_sleep_time = random.uniform(1, 5)
            time.sleep(random_sleep_time)  # IP 차단 방지용 랜덤한 시간 동안 대기

Processing songs: 100%|██████████| 100/100 [4:29:29<00:00, 161.69s/it] 


In [55]:
song_data

,title,artist,ganre,release_date,like_cnt,lylics
0,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜\n그 미움까지 그리운 걸까\n그토록 미웠던 끝 모습을 꺼내보고 말야\n그때 ...
1,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜\n그 미움까지 그리운 걸까\n그토록 미웠던 끝 모습을 꺼내보고 말야\n그때 ...
2,us,잭킹콩 (JKC),"R&B/Soul, 인디음악",2025.01.07,0,I’ll make sure you feel safe in my shoulders\n...
3,보통의 이야기,소보,"인디음악, 포크/블루스",2025.01.07,0,헝클어져있던 주머니속 실타래들\n널부러져있던 깨진 기억의 파편들\n상처들\n찾으려해...
4,엔딩샷 (Ending Shot),정가이 (와인루프),"발라드, 인디음악",2025.01.07,0,믿을 수 없어 네가 내 곁에 없단 게\n우린 죽도록 사랑했는데\n널 다시는 볼 수 ...
...,...,...,...,...,...,...
4996,Love for,요셉(Yoseph),"인디음악, 록/메탈",2025.01.06,0,What do you love for?\nI mean why?\nDo you lov...
4997,엄마,로키드 (Lokid),"발라드, 인디음악",2025.01.05,0,‘엄마 사랑해’라는 말은\n왜 이렇게 낯간지러운지\n세상에서 제일 소중한 내 편인데...
4998,바보같지,1DAY,"랩/힙합, 인디음악",2025.01.05,0,어? 시간이 이렇게나 됐어\n어쩌다 내 나이가 벌써\n난 아직 운전도 못하고\n직업...
4999,썩은그물,2단지,"인디음악, 포크/블루스",2025.01.05,0,어제의 꿈속이라면 건져내기 좋은데\n등 푸른 지옥이라도 달려오면 어쩌나\n강가에 비...


In [63]:
import datetime

#데이터 프레임 저장
now = datetime.datetime.now()
address = 'C:\\Two_Kim_and_One_Lee\\data\\'
# os.chdir("./data")
# song_data.to_csv(address, index=False, encoding='utf-8-sig')
song_data.to_csv(path_or_buf=address+'melon.csv')
# song_data.to_csv('melon{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

In [6]:
song_dt = pd.read_csv("./data/melon.csv")

In [7]:
song_dt

,Unnamed: 0,title,artist,ganre,release_date,like_cnt,lylics
0,0,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜\n그 미움까지 그리운 걸까\n그토록 미웠던 끝 모습을 꺼내보고 말야\n그때 ...
1,1,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜\n그 미움까지 그리운 걸까\n그토록 미웠던 끝 모습을 꺼내보고 말야\n그때 ...
2,2,us,잭킹콩 (JKC),"R&B/Soul, 인디음악",2025.01.07,0,I’ll make sure you feel safe in my shoulders\n...
3,3,보통의 이야기,소보,"인디음악, 포크/블루스",2025.01.07,0,헝클어져있던 주머니속 실타래들\n널부러져있던 깨진 기억의 파편들\n상처들\n찾으려해...
4,4,엔딩샷 (Ending Shot),정가이 (와인루프),"발라드, 인디음악",2025.01.07,0,믿을 수 없어 네가 내 곁에 없단 게\n우린 죽도록 사랑했는데\n널 다시는 볼 수 ...
...,...,...,...,...,...,...,...
4996,4996,Love for,요셉(Yoseph),"인디음악, 록/메탈",2025.01.06,0,What do you love for?\nI mean why?\nDo you lov...
4997,4997,엄마,로키드 (Lokid),"발라드, 인디음악",2025.01.05,0,‘엄마 사랑해’라는 말은\n왜 이렇게 낯간지러운지\n세상에서 제일 소중한 내 편인데...
4998,4998,바보같지,1DAY,"랩/힙합, 인디음악",2025.01.05,0,어? 시간이 이렇게나 됐어\n어쩌다 내 나이가 벌써\n난 아직 운전도 못하고\n직업...
4999,4999,썩은그물,2단지,"인디음악, 포크/블루스",2025.01.05,0,어제의 꿈속이라면 건져내기 좋은데\n등 푸른 지옥이라도 달려오면 어쩌나\n강가에 비...


In [7]:
song_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    5001 non-null   int64 
 1   title         5001 non-null   object
 2   artist        5001 non-null   object
 3   ganre         5001 non-null   object
 4   release_date  5001 non-null   object
 5   like_cnt      5001 non-null   int64 
 6   lylics        5001 non-null   object
dtypes: int64(2), object(5)
memory usage: 273.6+ KB


과오 바로 잡고 있습니다.. 나중에 소스 정리 할게요... ㅠㅠ 
* 과오 1. 줄바꿈 태그 안 없애고 저장 했음 (가사 열 )
* 과오 2. 가사 lyrics 행인데 lylics로 저장... (창피하니까 바꿀게요 ..)

In [8]:
# HTML 태그 제거 함수
def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

In [9]:
# 데이터프레임의 'lylics' 열에 적용
song_dt['lylics'] = song_dt['lylics'].apply(remove_html_tags)

# 줄 바꿈 문자 제거
song_dt['lyrics'] = song_dt['lylics'].str.replace('\n', '', regex=False)

In [10]:
song_dt.head()

,Unnamed: 0,title,artist,ganre,release_date,like_cnt,lylics,lyrics
0,0,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜\n그 미움까지 그리운 걸까\n그토록 미웠던 끝 모습을 꺼내보고 말야\n그때 ...,난 왜그 미움까지 그리운 걸까그토록 미웠던 끝 모습을 꺼내보고 말야그때 그렇게 있지...
1,1,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜\n그 미움까지 그리운 걸까\n그토록 미웠던 끝 모습을 꺼내보고 말야\n그때 ...,난 왜그 미움까지 그리운 걸까그토록 미웠던 끝 모습을 꺼내보고 말야그때 그렇게 있지...
2,2,us,잭킹콩 (JKC),"R&B/Soul, 인디음악",2025.01.07,0,I’ll make sure you feel safe in my shoulders\n...,I’ll make sure you feel safe in my shoulderswh...
3,3,보통의 이야기,소보,"인디음악, 포크/블루스",2025.01.07,0,헝클어져있던 주머니속 실타래들\n널부러져있던 깨진 기억의 파편들\n상처들\n찾으려해...,헝클어져있던 주머니속 실타래들널부러져있던 깨진 기억의 파편들상처들찾으려해 철이없던 ...
4,4,엔딩샷 (Ending Shot),정가이 (와인루프),"발라드, 인디음악",2025.01.07,0,믿을 수 없어 네가 내 곁에 없단 게\n우린 죽도록 사랑했는데\n널 다시는 볼 수 ...,믿을 수 없어 네가 내 곁에 없단 게우린 죽도록 사랑했는데널 다시는 볼 수 없다는 ...


In [11]:
song_dt = song_dt.drop(columns=['lylics'])

In [12]:
song_dt

,Unnamed: 0,title,artist,ganre,release_date,like_cnt,lyrics
0,0,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜그 미움까지 그리운 걸까그토록 미웠던 끝 모습을 꺼내보고 말야그때 그렇게 있지...
1,1,어떻게든 이별,참깨와 솜사탕,"발라드, 인디음악",2025.01.07,0,난 왜그 미움까지 그리운 걸까그토록 미웠던 끝 모습을 꺼내보고 말야그때 그렇게 있지...
2,2,us,잭킹콩 (JKC),"R&B/Soul, 인디음악",2025.01.07,0,I’ll make sure you feel safe in my shoulderswh...
3,3,보통의 이야기,소보,"인디음악, 포크/블루스",2025.01.07,0,헝클어져있던 주머니속 실타래들널부러져있던 깨진 기억의 파편들상처들찾으려해 철이없던 ...
4,4,엔딩샷 (Ending Shot),정가이 (와인루프),"발라드, 인디음악",2025.01.07,0,믿을 수 없어 네가 내 곁에 없단 게우린 죽도록 사랑했는데널 다시는 볼 수 없다는 ...
...,...,...,...,...,...,...,...
4996,4996,Love for,요셉(Yoseph),"인디음악, 록/메탈",2025.01.06,0,What do you love for?I mean why?Do you love me...
4997,4997,엄마,로키드 (Lokid),"발라드, 인디음악",2025.01.05,0,‘엄마 사랑해’라는 말은왜 이렇게 낯간지러운지세상에서 제일 소중한 내 편인데어른이 ...
4998,4998,바보같지,1DAY,"랩/힙합, 인디음악",2025.01.05,0,어? 시간이 이렇게나 됐어어쩌다 내 나이가 벌써난 아직 운전도 못하고직업 따윈 없어...
4999,4999,썩은그물,2단지,"인디음악, 포크/블루스",2025.01.05,0,어제의 꿈속이라면 건져내기 좋은데등 푸른 지옥이라도 달려오면 어쩌나강가에 비췄던 나...


In [13]:

#데이터 프레임 저장
address = 'C:\\Two_Kim_and_One_Lee\\data\\'
song_dt.to_csv(path_or_buf=address+'melon.csv')